In [29]:
from tkinter import *
import csv
import copy
import argparse
import itertools
from collections import Counter
from collections import deque
import  speech_recognition as sr

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.editor import *

from tkinter import filedialog
from tkVideoPlayer import TkinterVideo

import cv2 as cv
import numpy as np
import mediapipe as mp

from utils import CvFpsCalc
from model import KeyPointClassifier
from model import PointHistoryClassifier

import gtts
import os
from playsound import playsound

import datetime
import tkinter as tk
from tkinter import filedialog
from tkVideoPlayer import TkinterVideo


from tkinter import *

import numpy as np
from PIL import ImageFont, ImageDraw, Image
import cv2

import pyttsx3
engine = pyttsx3.init()
engine.setProperty('voice',"HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\MSTTS_V110_arEG_Hoda")


import arabic_reshaper
from bidi.algorithm import get_display
## Make canvas and set the color
img = np.zeros((200,400,3),np.uint8)

b,g,r,a = 0,255,0,0


## Use simsum.ttc to write عربى.
fontpath = "alfont_com_arial-1.ttf"
font = ImageFont.truetype(fontpath, 32)
img_pil = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)


labels = ["ا","ب","ت","ث","ج","ح","خ","د","ذ","ر","ز","س","ش","ص","ض","ط","ظ","ع","غ","ف","ق","ك","ل","م","ن","ه","و","ء","ة","ال","ي"," "]

In [30]:
def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("--device", type=int, default=0)
    parser.add_argument("--width", help='cap width', type=int, default=960)
    parser.add_argument("--height", help='cap height', type=int, default=540)

    parser.add_argument('--use_static_image_mode', action='store_true')
    parser.add_argument("--min_detection_confidence",
                        help='min_detection_confidence',
                        type=float,
                        default=0.7)
    parser.add_argument("--min_tracking_confidence",
                        help='min_tracking_confidence',
                        type=int,
                        default=0.5)
   # args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    return args

In [31]:
def main():
    import tkinter as tk
    global index
    global Word
    
    arr=""
    Word=""
    temp=[]
    arr2=[]
    c=0
    delayCount1=0
    delayCount2=0
    e=""
    wrd=""
    #var=StringVar()
    # Argument parsing #################################################################
    args = get_args()

    cap_device = args.device
    cap_width = args.width
    cap_height = args.height

    use_static_image_mode = args.use_static_image_mode
    min_detection_confidence = args.min_detection_confidence
    min_tracking_confidence = args.min_tracking_confidence

    use_brect = True

    # Camera preparation ###############################################################
    cap = cv.VideoCapture(cap_device)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)

    # Model load #############################################################
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=1,
        min_detection_confidence=min_detection_confidence,
        min_tracking_confidence=min_tracking_confidence,
    )

    keypoint_classifier = KeyPointClassifier()

    point_history_classifier = PointHistoryClassifier()

    # Read labels ###########################################################
    with open('model/keypoint_classifier/keypoint_classifier_label.csv',
              encoding='utf-8-sig') as f:
        keypoint_classifier_labels = csv.reader(f)
        keypoint_classifier_labels = [
            row[0] for row in keypoint_classifier_labels
        ]
    with open(
            'model/point_history_classifier/point_history_classifier_label.csv',
            encoding='utf-8-sig') as f:
        point_history_classifier_labels = csv.reader(f)
        point_history_classifier_labels = [
            row[0] for row in point_history_classifier_labels
        ]

    # FPS Measurement ########################################################
    cvFpsCalc = CvFpsCalc(buffer_len=10)

    # Coordinate history #################################################################
    history_length = 16
    point_history = deque(maxlen=history_length)

    # Finger gesture history ################################################
    finger_gesture_history = deque(maxlen=history_length)

    #  ########################################################################
    mode = 0
    
    while True:
        fps = cvFpsCalc.get()

        # Process Key (ESC: end) #################################################
        key = cv.waitKey(10)
        if key == 27:  # ESC
            break
        number, mode = select_mode(key, mode)

        # Camera capture #####################################################
        ret, image = cap.read()
        if not ret:
            break
        image = cv.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        # Detection implementation #############################################################
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True

        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                  results.multi_handedness):
                # Bounding box calculation
                brect = calc_bounding_rect(debug_image, hand_landmarks)
                # Landmark calculation
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)
                pre_processed_point_history_list = pre_process_point_history(
                    debug_image, point_history)
                # Write to the dataset file
                logging_csv(number, mode, pre_processed_landmark_list,
                            pre_processed_point_history_list)

                # Hand sign classification
                hand_sign_id = keypoint_classifier(pre_processed_landmark_list)
                if hand_sign_id == "Not applicable":  # Point gesture
                    point_history.append(landmark_list[8])
                else:
                    point_history.append([0, 0])

                # Finger gesture classification
                finger_gesture_id = 0
                point_history_len = len(pre_processed_point_history_list)
                if point_history_len == (history_length * 2):
                    finger_gesture_id = point_history_classifier(
                        pre_processed_point_history_list)

                # Calculates the gesture IDs in the latest detection
                finger_gesture_history.append(finger_gesture_id)
                most_common_fg_id = Counter(
                    finger_gesture_history).most_common()

                # Drawing part
                debug_image = draw_bounding_rect(use_brect, debug_image, brect)
                debug_image = draw_landmarks(debug_image, landmark_list)
                debug_image = draw_info_text(
                    debug_image,
                    brect,
                    handedness,
                    keypoint_classifier_labels[hand_sign_id],
                    point_history_classifier_labels[most_common_fg_id[0][0]],
                )
                
                import tkinter as tk                
                if (hand_sign_id >=0 and hand_sign_id < 32):
                    e=''.join(labels[hand_sign_id])
                    Word=Word+labels[hand_sign_id]      
                    if  e ==labels[hand_sign_id]:
                        c=c+1
                        if(c==20):
                            arr+=e
                            c=0
                    else:
                        arr-=arr.join(e)
                        arr=''.join(map(str, [v for i, v in enumerate(arr) if i == 0 or v != arr[i-2]]))
                            
                elif (hand_sign_id == 32):
                    if(delayCount1<=10):
                        delayCount1=delayCount1+1
                    else:
                        
                        arr=arr[:-1]
                        cv.waitKey(700)
                        delayCount1=0
                        var.set(arr)
                        T.update_idletasks()
                        
                elif (hand_sign_id == 33):
                    if(delayCount2<=20):
                        delayCount2=delayCount2+1
                    else:
                        tts = gtts.gTTS(arr, lang="ar")
                        tts.save("h3.mp3")
                        playsound("h3.mp3")
                        os.remove('h3.mp3') 
                        delayCount2=0
                print(arr)
                print (Word)
                arr=arr
                var.set(arr)
                T.update()
                cv.waitKey(50)
        
        else:
            point_history.append([0, 0])
       
        debug_image = draw_point_history(debug_image, point_history)
        debug_image = draw_info(debug_image, fps, mode, number)

        # Screen reflection #############################################################
        cv.imshow('Hand Gesture Recognition', debug_image)
         
        T.pack
       
    cap.release()
    cv.destroyAllWindows()


def select_mode(key, mode):
    number = -1
    if 48 <= key <= 57:  # 0 ~ 9
        number = key - 48
    if key == 110:  # n
        mode = 0
    if key == 107:  # k
        mode = 1
    if key == 104:  # h
        mode = 2
    return number, mode


def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


def pre_process_point_history(image, point_history):
    image_width, image_height = image.shape[1], image.shape[0]

    temp_point_history = copy.deepcopy(point_history)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, point in enumerate(temp_point_history):
        if index == 0:
            base_x, base_y = point[0], point[1]

        temp_point_history[index][0] = (temp_point_history[index][0] -
                                        base_x) / image_width
        temp_point_history[index][1] = (temp_point_history[index][1] -
                                        base_y) / image_height

    # Convert to a one-dimensional list
    temp_point_history = list(
        itertools.chain.from_iterable(temp_point_history))

    return temp_point_history


def logging_csv(number, mode, landmark_list, point_history_list):
    if mode == 0:
        pass
    if mode == 1 and (0 <= number <= 34):
        csv_path = 'model/keypoint_classifier/keypoint.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number+30, *landmark_list])
    if mode == 2 and (0 <= number <= 34):
        csv_path = 'model/point_history_classifier/point_history.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *point_history_list])
    return


def draw_landmarks(image, landmark_point):
    if len(landmark_point) > 0:
        # Thumb
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (150, 216, 242), 2)

        # Index finger
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (150, 216, 242), 2)

        # Middle finger
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (150, 216, 242), 2)

        # Ring finger
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (150, 216, 242), 2)

        # Little finger
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (150, 216, 242), 2)

        # Palm
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (150, 216, 242), 2)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (150, 216, 242), 2)

    # Key Points
    for index, landmark in enumerate(landmark_point):
        if index == 0:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 1:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 2:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 3:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 4:  
            cv.circle(image, (landmark[0], landmark[1]), 8, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 5:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 6:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 7:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 8:  
            cv.circle(image, (landmark[0], landmark[1]), 8, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 9:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 10:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 11:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 12:  
            cv.circle(image, (landmark[0], landmark[1]), 8, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 13: 
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 14:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 15: 
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 16: 
            cv.circle(image, (landmark[0], landmark[1]), 8, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 17:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 18:  
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 19: 
            cv.circle(image, (landmark[0], landmark[1]), 5, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 20: 
            cv.circle(image, (landmark[0], landmark[1]), 8, (75, 84, 188),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)

    return image


def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (59, 68, 121), 1)

    return image


def draw_info_text(image, brect, handedness, hand_sign_text,
                   finger_gesture_text):
    cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22),
                 (59, 68, 121), -1)

    info_text = handedness.classification[0].label[0:]
    if hand_sign_text != "":
        info_text = info_text + ':' + hand_sign_text
    cv.putText(image, info_text, (brect[0] + 5, brect[1] - 4),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)

    if finger_gesture_text != "":
        cv.putText(image, "Finger Gesture:" + finger_gesture_text, (10, 60),
                   cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
        cv.putText(image, "Finger Gesture:" + finger_gesture_text, (10, 60),
                   cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2,
                   cv.LINE_AA)

    return image


def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv.circle(image, (point[0], point[1]), 1 + int(index / 2),
                      (152, 251, 152), 2)

    return image


def draw_info(image, fps, mode, number):
    cv.putText(image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX,
               1.0, (0, 0, 0), 4, cv.LINE_AA)
    cv.putText(image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX,
               1.0, (255, 255, 255), 2, cv.LINE_AA)

    mode_string = ['Logging Key Point', 'Logging Point History']
    if 1 <= mode <= 2:
        cv.putText(image, "MODE:" + mode_string[mode - 1], (10, 90),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                   cv.LINE_AA)
        if 0 <= number <= 34:
            cv.putText(image, "NUM:" + str(number), (10, 110),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                       cv.LINE_AA)
    return image

In [32]:
def vedioSign():
        r=sr.Recognizer()
        with sr.Microphone() as src:
            print('Say something....')
            audio=r.listen(src)
            t=r.recognize_google(audio,language='ar-AR')
        
        arr = list(t)
        global T2
        T2.destroy()
        t=''.join(arr)
        var = StringVar()    # create a string variable
        var.set(t)      # set it to "letter"
        T2= Label(root, textvariable = var,font =font2,fg='black',bg='#f2d896')
        T2.place(relx=0.6, rely=0.150, anchor=NE)      
        labels = ["ا","ب","ت","ث","ج","ح","خ","د","ذ","ر","ز","س","ش","ص","ض","ط","ظ","ع","غ","ف","ق","ك","ل","م","ن","ه","و","ء","ة","ال","لا","ى"," "]

        for index in range(len(arr)) :
            if arr[index]==' ':
                arr[index]='1'
            elif arr[index]=='ا':
                arr[index]='2'
            elif arr[index]=='ب':
                arr[index]='3'
            elif arr[index]=='ت':
                arr[index]='4'
            elif arr[index]=='ث':
                arr[index]='5'
            elif arr[index]=='ج':
                arr[index]='6'
            elif arr[index]=='ح':
                arr[index]='7'
            elif arr[index]=='خ':
                arr[index]='8'
            elif arr[index]=='د':
                arr[index]='9'
            elif arr[index]=='ذ':
                arr[index]='10'
            elif arr[index]=='ر':
                arr[index]='11'
            elif arr[index]=='ز':
                arr[index]='12'
            elif arr[index]=='س':
                arr[index]='13'
            elif arr[index]=='ش':
                arr[index]='14'
            elif arr[index]=='ص':
                arr[index]='15'
            elif arr[index]=='ض':
                arr[index]='16'
            elif arr[index]=='ط':
                arr[index]='17'
            elif arr[index]=='ظ':
                arr[index]='18'
            elif arr[index]=='ع':
                arr[index]='19'
            elif arr[index]=='غ':
                arr[index]='20'
            elif arr[index]=='ف':
                arr[index]='21'
            elif arr[index]=='ق':
                arr[index]='22'
            elif arr[index]=='ك':
                arr[index]='23'
            elif arr[index]=='ل':
                arr[index]='24'
            elif arr[index]=='م':
                arr[index]='25'
            elif arr[index]=='ن':
                arr[index]='26'
            elif arr[index]=='ه':
                arr[index]='27'
            elif arr[index]=='و':
                arr[index]='28'
            elif arr[index]=='ء':
                arr[index]='29'
            elif arr[index]=='ة':
                arr[index]='30'
            elif arr[index]=='ي':
                arr[index]='31'
            elif arr[index]=='ئ':
                arr[index]='33'
            elif arr[index]=='ؤ':
                arr[index]='34'

        video=[]
        for i in range(len(arr)) : 
            video.append(ImageClip('%s.jpg' %str(arr[i])).set_duration(2))
        final_clip = concatenate_videoclips(video, method='compose')
        final_clip.write_videofile("merge.mp4", fps=12, remove_temp=True, codec="libx264", audio_codec="aac")

        """ pauses and plays """
        vid_player.load('merge.mp4')
        vid_player.play()
         


def showvedio():
    cap = cv2.VideoCapture('merge.mp4')
    if (cap.isOpened()== False):
        print("Error opening video file")

    # Read until video is completed
    while(cap.isOpened()):

    # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
      # Display the resulting frame
            cv2.imshow('Frame', frame)

      # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Break the loop
        else:
            break

    # When everything done, release
    # the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    os.remove('merge.mp4')

In [33]:
def update_duration(event):
    """ updates the duration after finding the duration """
    end_time["text"] = str(datetime.timedelta(seconds=vid_player.duration()))
    """ updates the scale value """
    """ used to seek a specific timeframe """
def play_pause():
    """ pauses and plays """
    vid_player.load('merge.mp4')
    if vid_player.is_paused():
        vid_player.play()

    else:
        vid_player.pause()


def video_ended(event):
    """ handle video ended """
    play_pause_btn["text"] = "Play"




In [34]:
from tkVideoPlayer import TkinterVideo
import tkinter as tk
from tkinter import font
global arr
from tkinter import *
from tkinter.ttk import *
from tkinter import *   
from tkinter import *
from threading import Thread
from PIL import Image, ImageTk

arr=""
top=tk.Tk()#for create frame
top.title("Detect Sign Hand")
top.geometry("1200x1000")
#top.configure(bg='#FFEBC1')

top.configure(bg='#f2d896')
top.attributes('-fullscreen', True)


new=Toplevel(top) 
new.withdraw()#to hide frame
new.geometry("1200x1000")
new.configure(bg='#f2d896')
new.attributes('-fullscreen', True)

root=Toplevel(top)
root.withdraw()#to hide frame
root.geometry("1200x1000")
root.configure(bg='#f2d896')
root.attributes('-fullscreen', True)


guideFrame=Toplevel(top)
guideFrame.withdraw()#to hide frame
guideFrame.geometry("1200x1000")
guideFrame.configure(bg='#f2d896')
guideFrame.attributes('-fullscreen', True)

p1 = PhotoImage(file = '123.png')
root.iconphoto(False, p1)
new.iconphoto(False, p1)
top.iconphoto(False, p1)
guideFrame.iconphoto(False, p1)

# Entry widget
my_text1 = Entry(top,width=25,justify=CENTER, background="#79443B",foreground='WHITE', font=('Droid Arabic Kufi', 50,'bold'), borderwidth=2,relief=RAISED )
my_text1.insert(0, "Hand Sign Language ")
my_text1.pack(padx=50, pady=20)
my_text2 = Entry(new, width=30, justify=CENTER, background="#79443B",foreground='white', font=('Droid Arabic Kufi', 50,'bold'), borderwidth=2,relief=RAISED)
my_text2.insert(0, "Signature Hand Gestures")
my_text2.pack(padx=50, pady=20)

my_text3 = Entry(root, width=30, justify=CENTER, background="#79443B",foreground='white', font=('Droid Arabic Kufi', 50,'bold'), borderwidth=2,relief=RAISED)
my_text3.insert(0, "Converting Speech Into A Sign")
my_text3.pack(padx=50, pady=20)

my_text4 = Entry(guideFrame, width=30, justify=CENTER, background="#79443B",foreground='white', font=('Droid Arabic Kufi', 50,'bold'), borderwidth=2,relief=RAISED)
my_text4.insert(0, "Sign language in Arabic")
my_text4.pack(padx=50, pady=20)



def change_to_cam():
        top.withdraw()
        new.deiconify()# to show frame
        new.title("Detect Sign Hand")
        
        bt = Button(new, text="open camera",  bg='#BC544B', fg='black', command=main,height= 1, width=10)
        bt['font'] = font2
        bt.place(relx = 0.7, rely=0.3 , anchor = W)
        
        def backFrame():
            bt.destroy()
            b9.destroy()
            top.deiconify()
            new.withdraw()
        b9 = Button(new, text="Back",  bg='#BC544B', fg='#1F201F', command=backFrame,height= 1, width=10)
        b9['font'] = font2
        b9.place(relx = 0.7, rely=0.7 , anchor = W)
         
def change_to_speak():
        top.withdraw()
        root.deiconify()# to show frame
        root.title("Detect Sign Hand")
        load_btn = Button(root, text="Speak", bg='#BC544B', fg='black',  command=vedioSign,height= 1, width=10)
        load_btn['font'] = font2
        load_btn.place(relx=0.9, rely=0.3, anchor=NE)
        play_pause_btn = tk.Button(root,text="repeat", bg='#BC544B', fg='black',command=play_pause,height= 1, width=10)
        play_pause_btn['font'] = font2
        play_pause_btn.place(relx=0.9, rely=0.5, anchor=NE)
        def backFrame():
            load_btn.destroy()
            b4.destroy()
            play_pause_btn.destroy()
            top.deiconify()
            root.withdraw()
        b4 = Button(root, text="Back",  bg='#BC544B', fg='#1F201F', command=backFrame,height= 1, width=10)
        b4['font'] = font2
        b4.place(relx=0.9, rely=0.7, anchor=NE)

def change_to_guide():
        top.withdraw()
        guideFrame.deiconify()# to show frame
        guideFrame.title("Detect Sign Hand")

        def backFrame():
            b7.destroy()
            top.deiconify()
            guideFrame.withdraw()
        b7 = Button(guideFrame, text="Back",  bg='#BC544B', fg='#1F201F', command=backFrame,height= 1, width=10)
        b7['font'] = font2
        b7.place(relx=0.9, rely=0.6, anchor=NE)

def change_text():
      var.set("")
        
# Create fonts for making difference in the frame
font1 = font.Font(family='Georgia', size='35', weight='bold')
font2 = font.Font(family='Aerial', size='45')



img =Image.open('T.jpg')
resized = img.resize((300, 800), Image.ANTIALIAS)
bg = ImageTk.PhotoImage(resized)
labelm = Label(top, image=bg,borderwidth=0)
labelm.place(relx=0.0, rely=0.1, anchor=NW)
labelm1 = Label(top, image=bg,borderwidth=0)
labelm1.place(relx=1, rely=0.1, anchor=NE)



img2 =Image.open('st3.jpg')
resized2 = img2.resize((800, 680), Image.ANTIALIAS)
bg2 = ImageTk.PhotoImage(resized2)
labelm2 = Label(guideFrame, image=bg2,borderwidth=0)
labelm2.place(relx=0.0, rely=0.2, anchor=NW)

img3 =Image.open('l.jpg')
resized3 = img3.resize((150, 150), Image.ANTIALIAS)
bg3 = ImageTk.PhotoImage(resized3)
labelm3 = Label(top, image=bg3,borderwidth=0)
labelm3.place(relx=0.0, rely=0.0, anchor=NW)
labelm3 = Label(new, image=bg3,borderwidth=0)
labelm3.place(relx=0.0, rely=0.0, anchor=NW)
labelm3 = Label(root, image=bg3,borderwidth=0)
labelm3.place(relx=0.0, rely=0.0, anchor=NW)
labelm3 = Label(guideFrame, image=bg3,borderwidth=0)
labelm3.place(relx=0.0, rely=0.0, anchor=NW)

img4 =Image.open('lo.jpg')
resized4 = img4.resize((150, 150), Image.ANTIALIAS)
bg4 = ImageTk.PhotoImage(resized4)
labelm5 = Label(top, image=bg4,borderwidth=0)
labelm5.place(relx=0.9, rely=0.0, anchor=NW)
labelm5 = Label(new, image=bg4,borderwidth=0)
labelm5.place(relx=0.9, rely=0.0, anchor=NW)
labelm5 = Label(root, image=bg4,borderwidth=0)
labelm5.place(relx=0.9, rely=0.0, anchor=NW)
labelm5 = Label(guideFrame, image=bg4,borderwidth=0)
labelm5.place(relx=0.9, rely=0.0, anchor=NW)


btn1 = Button(top, text="Scan Hand Sign", bg='#BC544B', fg='black',command=change_to_cam,height= 1, width=15)
btn1['font'] = font2
btn1.place(relx=0.5, rely=0.3, anchor=CENTER)
btn2 = Button(top, text="Speak", bg='#BC544B', fg='black',  command=change_to_speak,height= 1, width=15)
btn2['font'] = font2
btn2.place(relx=0.5, rely=0.5, anchor=CENTER)

btn3 = Button(top, text="Signal guide", bg='#BC544B', fg='black',command=change_to_guide,height= 1, width=15)
btn3['font'] = font2
btn3.place(relx=0.5, rely=0.7, anchor=CENTER)

exit_button = Button(top, text="EXIT",bg='#BC544B', fg='#800517', command=top.destroy,height= 1, width=15)
exit_button['font'] = font2
exit_button.place(relx=0.5, rely=0.9, anchor=CENTER)

T2=Label(root)

btn4=Button(new, text="Change Text!", bg='#BC544B', fg='black',command=change_text,height= 1, width=10)
btn4['font'] = font2
btn4.place(relx = 0.7, rely=0.5, anchor = W)      


var = StringVar()    # create a string variable
var.set(arr)      # set it to "letter"
T= Label(new,textvariable = var,font =font2,fg='black',bg='#f2d896')   # display var "letter" as l2
T.place(relx=0.6, rely=0.3, anchor=NE)      
 


vid_player = TkinterVideo(scaled=True, master=root)
vid_player.place(bordermode=OUTSIDE,relx = 0.1, rely=0.6 , anchor = W,height=600,width=800)


In [35]:
top.mainloop()